In [2]:
# Đề tài dự đoán tuổi giới tính và hình dang khuôn mặt
# Tải các thư viện cần thiết
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
#Load thư viện hình ảnh UTKface gồm hơn 20k ảnh có độ tuổi từ 1-120
Data = '../input/utkface-new/UTKFace/'

In [4]:
#  Xử lý dữ liệu - Chèn thêm cột Tuổi và Giới tính để tạo ra 1 dataframe
image_paths = []
age_labels = []
gender_labels = []
for filename in tqdm(os.listdir(Data)):
    image_path = os.path.join(Data, filename)
    temp = filename.split('_')
    age = int(temp[0])
    gender = int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)
df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
df.head(21)

In [5]:
# Gán ý nghĩa của giá trị 0 và 1 của gender    
gender_value = {0:'Nam', 1:'Nữ'}

In [6]:
# Dữ liệu từ hình ảnh của thư viện
sns.countplot(df['gender'])

In [7]:
sns.distplot(df['age'])

In [8]:
df=df[df['age']<90]

sns.countplot(df['age'])

In [9]:
sns.distplot(df['age'])

In [10]:
df=df[df['age']<130]
img = Image.open(df['image'][10])
plt.imshow(img);

In [11]:
# Chiết xuất nét đặt trưng của ảnh
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = np.array(img)
        features.append(img)
        
    features = np.array(features)
    return features
x = extract_features(df['image'])

In [12]:
#Scale ảnh và chuẩn bị dữ liệu train test
x = x/255.0
y_gender = np.array(df['gender'])
y_age = np.array(df['age'])
x_train_age,x_test_age,y_train_age,y_test_age= train_test_split(x, y_age, test_size =0.2, stratify=y_age)
x_train_gender,x_test_gender,y_train_gender,y_test_gender= train_test_split(x, y_gender, test_size =0.2, stratify=y_gender)
input_shape = (128, 128, 1)

In [13]:
# Xây dựng model dự đoán tuổi và giới tính
inputs = Input((input_shape))
convolutional_1 = Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxpooling_1 = MaxPooling2D(pool_size=(2, 2)) (convolutional_1)

convolutional_2 = Conv2D(64, kernel_size=(3, 3), activation='relu') (maxpooling_1)
maxpooling_2 = MaxPooling2D(pool_size=(2, 2)) (convolutional_2)

convolutional_3 = Conv2D(128, kernel_size=(3, 3), activation='relu') (maxpooling_2)
maxpooling_3 = MaxPooling2D(pool_size=(2, 2)) (convolutional_3)

convolutional_4 = Conv2D(256, kernel_size=(3, 3), activation='relu') (maxpooling_3)
maxpooling_4 = MaxPooling2D(pool_size=(2, 2)) (convolutional_4)

convolutional_5 = Conv2D(512, kernel_size=(3, 3), activation='relu') (maxpooling_4)
maxpooling_5 = MaxPooling2D(pool_size=(2, 2)) (convolutional_5)

flatten = Flatten() (maxpooling_5)

# Kết Nối layer
dense_1 = Dense(1024, activation='relu') (flatten)
dense_2 = Dense(512, activation='relu') (flatten)
dropout_1 = Dropout(0.5) (dense_1)
dropout_2 = Dropout(0.5) (dense_2)
output_1 = Dense(1, activation='sigmoid', name='gender_out') (dropout_1)
output_2 = Dense(1, activation='relu', name='age_out') (dropout_2)
model = Model(inputs=[inputs], outputs=[output_1, output_2])
model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics=['accuracy'])

In [14]:
# train model
detect_age_gender = model.fit(x=x, y=[y_gender, y_age], batch_size=32, epochs=40, validation_split=0.2)

In [15]:
# Vẽ đồ thị kết quả của model sau khi train được
acc = detect_age_gender.history['gender_out_accuracy']
val_acc = detect_age_gender.history['val_gender_out_accuracy']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'p', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()
loss = detect_age_gender.history['gender_out_loss']
val_loss = detect_age_gender.history['val_gender_out_loss']
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'p', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()

In [18]:
#Test dự đoán tuổi và giới tính
image_index = 300
print("Giới tính đúng:", gender_value[y_gender[image_index]], "      Tuổi đúng:", y_age[image_index])
pred = model.predict(x[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_value[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])
print("Giới tính dự đoán:", pred_gender, "   Tuổi dự đoán:", pred_age)
plt.axis('off')
plt.imshow(x[image_index].reshape(128, 128));

In [22]:
image_index = 3000
print("Giới tính đúng:", gender_value[y_gender[image_index]], "      Tuổi đúng:", y_age[image_index])
pred = model.predict(x[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_value[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])
print("Giới tính dự đoán:", pred_gender, "   Tuổi dự đoán:", pred_age)
plt.axis('off')
plt.imshow(x[image_index].reshape(128, 128));

In [23]:
image_index = 3040
print("Giới tính đúng:", gender_value[y_gender[image_index]], "      Tuổi đúng:", y_age[image_index])
pred = model.predict(x[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_value[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])
print("Giới tính dự đoán:", pred_gender, "   Tuổi dự đoán:", pred_age)
plt.axis('off')
plt.imshow(x[image_index].reshape(128, 128));

In [24]:
image_index = 8756
print("Giới tính đúng:", gender_value[y_gender[image_index]], "      Tuổi đúng:", y_age[image_index])
pred = model.predict(x[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_value[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])
print("Giới tính dự đoán:", pred_gender, "   Tuổi dự đoán:", pred_age)
plt.axis('off')
plt.imshow(x[image_index].reshape(128, 128));

In [25]:
#Load thư viện cho moddel nhận diện khuôn mặt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.optimizers import Adam, SGD
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import ImageFile
from keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [26]:
# Tạo tập test và tập train
im_shape=(200,200)
traindata ='../input/face-shape-dataset/FaceShape Dataset/training_set'    
testdata ='../input/face-shape-dataset/FaceShape Dataset/testing_set'       
seed = 10
BATCH_SIZE=32


In [27]:
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_data_generator = ImageDataGenerator( rescale=1./255, validation_split=0.2)

In [29]:
#Generator data train
train_generator=data_generator.flow_from_directory(traindata, target_size=im_shape, shuffle=True, seed=seed,
                                               class_mode="categorical",batch_size=BATCH_SIZE,subset="training")
#Generator data validation
validation_generator=val_data_generator.flow_from_directory(traindata,target_size=im_shape,shuffle=False,seed=seed,
                                               class_mode='categorical',batch_size=BATCH_SIZE,subset="validation")
              
#Generator data test
test_generator=ImageDataGenerator(rescale=1./255)
test_generator=test_generator.flow_from_directory(testdata,target_size=im_shape,shuffle=False,seed=seed,
                                               class_mode="categorical",batch_size=BATCH_SIZE)
nb_train_samples=train_generator.samples
nb_validation_samples=validation_generator.samples
nb_test_samples=test_generator.samples
classes=list(train_generator.class_indices.keys())
print(str(classes))
num_classes=len(classes)

In [30]:
#Xây dựng model
face_shape_model=Sequential()
face_shape_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(im_shape[0],im_shape[1],3)))
face_shape_model.add(MaxPooling2D(pool_size=(2,2)))
face_shape_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
face_shape_model.add(MaxPooling2D(pool_size=(2,2)))
face_shape_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
face_shape_model.add(MaxPooling2D(pool_size=(2,2)))
face_shape_model.add(Flatten())
face_shape_model.add(Dense(256,activation='relu'))
face_shape_model.add(Dropout(0.2))
face_shape_model.add(Dense(num_classes,activation='sigmoid'))
face_shape_model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics=['accuracy'])
face_shape_model.summary()

In [31]:
#Training
history_face_shape=face_shape_model.fit(train_generator,steps_per_epoch=nb_train_samples // BATCH_SIZE,epochs=40,validation_data=validation_generator,verbose=1,validation_steps=nb_validation_samples // BATCH_SIZE)

In [33]:
def predict_shape_face(file):
   img=load_img(file,target_size=(200,200))
   plt.imshow(img)
   img=img_to_array(img)
   img=img.reshape(1,200,200,3)
   img=img.astype('float32')
   img/=255
   np.argmax(face_shape_model.predict(img),axis=1)[0]
   print(classes[np.argmax(face_shape_model.predict(img),axis=1)[0]])



In [34]:
predict_shape_face('../input/utkface-new/crop_part1/10_0_0_20170110220514186.jpg.chip.jpg')

In [35]:
predict_shape_face('../input/face-shape-dataset/FaceShape Dataset/testing_set/Round/round (161).jpg')

In [36]:
predict_shape_face('../input/face-shape-dataset/FaceShape Dataset/testing_set/Oblong/oblong (10).jpg')

In [37]:
predict_shape_face('../input/face-shape-dataset/FaceShape Dataset/testing_set/Heart/heart (226).jpg')

In [38]:
predict_shape_face('../input/face-shape-dataset/FaceShape Dataset/testing_set/Square/square (147).jpg')